# Operations Research 3: Examples for Theory

This code is for students who take the class Operations Research. Students should finish the installation of Gurobi and Python before workersed and make sure an academic liscense for Gurobi is applied and activated.

We introduce an example for Network flow problem in order to help students understand how to implement theories introduced in lectures with codes. More instruction is provided in the lecture video.

# Network Flow Problem
## Example: Assignment Problem

Import instance of an assignment problem. \
In the file, we have the arcs of a bipartite graph. Each arc means if the job $i$ is assigned to worker $j$, we should pay the cost $c_{ij}$. \
Each worker and job should be assigned no greater than one time in an assignment problem.

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
graph_info = pd.read_excel('bipartite_arcs_set.xlsx', 'graph')

In [3]:
print(graph_info)

   jobs workers  cost
0     A       E     2
1     A       F  9999
2     A       G    10
3     A       H     7
4     B       E  9999
5     B       F     4
6     B       G     3
7     B       H  9999
8     C       E     8
9     C       F     2
10    C       G     5
11    C       H  9999
12    D       E     7
13    D       F  9999
14    D       G     1
15    D       H     6


Prepare data into different format to make us construct model more efficiently.

In [4]:
# put arcs into a dictionary

flow_dict = {}

for i in range(len(graph_info)):
    flow_dict[(graph_info['jobs'][i], graph_info['workers'][i])] = graph_info['cost'][i]

In [5]:
# get workers and jobs sets

workers_spot = set(np.unique(graph_info['workers']))
jobs_spot = set(np.unique(graph_info['jobs']))

In [6]:
# seperate arcs and cost

arcs, cost = multidict(flow_dict)

After preparing data, we construct a model to solve the assignment problem.

In [7]:
eg3 = Model('Assignment')


# add variables (arcs)
x = {}
for arc in arcs:
    x[arc] = eg3.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'x' + arc[0] + arc[1])


# set objective value
eg3.setObjective(quicksum(x[arc] * cost[arc] for arc in arcs), GRB.MINIMIZE)



# each worker and job could be select no less than one time
for job in jobs_spot:
    arcs_list = []
    for arc in arcs:
        if arc[0] == job:
            arcs_list.append(x[arc])
        else:
            pass   
    eg3.addConstr(quicksum(arcs_list) == 1, name = 'job' + job)

    
for worker in workers_spot:
    arcs_list = []
    for arc in arcs:
        if arc[1] == worker:
            arcs_list.append(x[arc])
        else:
            pass
    eg3.addConstr(quicksum(arcs_list) == 1, name = 'worker' + worker)


eg3.optimize()


# Print solution
for var in eg3.getVars():
    print(var.varName, '=', var.x)
print("objective value =", eg3.objVal)

Using license file /Users/jennytzeng/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0xda4ae30c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 8 rows, 16 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   2.000000e+00   0.000000e+00      0s
       2    1.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  1.300000000e+01
xAE = 1.0
xAF = 0.0
xAG = 0.0
xAH = 0.0
xBE = 0.0
xBF = 0.0
xBG = 1.0
xBH = 0.0
xCE = 0.0
xCF = 1.0
xCG = 0.0
xCH = 0.0
xDE = 0.0
xDF = 0.0
xDG = 0.0
xDH = 1.0
objective value = 13.0
